<a href="https://colab.research.google.com/github/tongnet/fin7047_2026spr/blob/main/hw2_key" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#4 You spent USD1000 each on Jan 2, 2020 to buy Bitcoin (BTC) and Ripple (XRP) and hold both securities indefinitely. Using information from Yahoo Finance, answer the following questions:
#A) Get their prices on December 31, 2025.from Yahoo Finance using codes attached in the box on the next page.
#B) How much do you have December 31, 2025?
#C) What is the return of your investment?
#D) Which investment has better performance, BTC or XRP?

import yfinance as yf

tickers = ["BTC-USD", "XRP-USD"]

data = yf.download(
    tickers,
    start="2020-01-02",
    end="2026-01-01"   # end is exclusive
)

prices_20200102 = data["Close"].loc["2020-01-02"]
prices_20251231 = data["Close"].loc["2025-12-31"]

print("Close prices on 2020-01-02:")
print(prices_20200102)

print("\nClose prices on 2025-12-31:")
print(prices_20251231)




/tmp/ipython-input-4258338625.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  2 of 2 completed

Close prices on 2020-01-02:
Ticker
BTC-USD    6985.470215
XRP-USD       0.188043
Name: 2020-01-02 00:00:00, dtype: float64

Close prices on 2025-12-31:
Ticker
BTC-USD    87508.828125
XRP-USD        1.839973
Name: 2025-12-31 00:00:00, dtype: float64


In [2]:
# 5. Using Yahoo Finance data, download daily closing prices for Ethereum (ETH-USD) and the S&P 500 Index (^GSPC) over the past 12 months.
# A.	Compute daily returns for ETH and for the S&P 500 using percentage changes in closing prices.

import yfinance as yf
import pandas as pd

# Define tickers
tickers = ["ETH-USD", "^GSPC"]

# Specify date range
start_date = "2025-01-23"
end_date   = "2026-01-23"

# Download daily closing prices
data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    interval="1d"
)["Close"]

# Rename columns for clarity
data.columns = ["ETH", "SP500"]

# Compute daily percentage returns (keep NaNs)
returns = data.pct_change()

# Estimate standard deviation of daily returns
std_dev = returns.std()

# Display results
print("Sample period:", start_date, "to", end_date)
print(returns.head())
print(returns.tail())
print(std_dev)

/tmp/ipython-input-532609525.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  2 of 2 completed

Sample period: 2025-01-23 to 2026-01-23
                 ETH     SP500
Date                          
2025-01-23       NaN       NaN
2025-01-24 -0.007545 -0.002855
2025-01-25  0.002331  0.000000
2025-01-26 -0.024458  0.000000
2025-01-27 -0.017680 -0.014581
                 ETH     SP500
Date                          
2026-01-18 -0.008373  0.000000
2026-01-19 -0.028812  0.000000
2026-01-20 -0.078772 -0.020627
2026-01-21  0.014748  0.011588
2026-01-22 -0.009825  0.005488
ETH      0.039063
SP500    0.009739
dtype: float64



/tmp/ipython-input-532609525.py:26: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change()


In [9]:
# 6. Generate a dataset containing daily returns for Bitcoin BTC-USD) and the S&P 500 (^GSPC) from January 1, 2020 to December 31, 2025. Then analyze their relationship using a regression model, splitting the sample into 70% training and 30% testing sets.
# A. prepare data
import yfinance as yf
import pandas as pd
import numpy as np

# --------------------------------
# specify dates
# --------------------------------
start_date = "2020-01-01"
end_date   = "2025-12-31"

# --------------------------------
# Download price data
# --------------------------------
tickers = ["BTC-USD", "^GSPC"]
prices = yf.download(tickers, start=start_date, end=end_date)["Close"]

# --------------------------------
# Compute daily returns
# --------------------------------
returns = (prices / prices.shift(1)).dropna()
returns.columns = ["BTC_Return", "SP500_Return"]

returns.head()

# B: split the data
from sklearn.model_selection import train_test_split

X = returns[["SP500_Return"]]
y = returns["BTC_Return"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, shuffle=False
)

# C. Perform analysis using the training data
import statsmodels.api as sm

# Add intercept
X_train_const = sm.add_constant(X_train)
X_test_const  = sm.add_constant(X_test)

# Fit model
model = sm.OLS(y_train, X_train_const).fit()
print("summary statistics from training data")
print(model.summary())

# D. Use test data (prediction + evaluation)
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model.predict(X_test_const)

test_r2 = r2_score(y_test, y_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("summary statistics from test data")
print("\n--- Test performance ---")
print(f"Test R^2  : {test_r2:.4f}")
print(f"Test RMSE : {test_rmse:.6f}")




/tmp/ipython-input-2756846922.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start=start_date, end=end_date)["Close"]
[*********************100%***********************]  2 of 2 completed

summary statistics from training data
                            OLS Regression Results                            
Dep. Variable:             BTC_Return   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.165
Method:                 Least Squares   F-statistic:                     163.6
Date:                Fri, 23 Jan 2026   Prob (F-statistic):           2.74e-34
Time:                        22:15:30   Log-Likelihood:                 1614.5
No. Observations:                 825   AIC:                            -3225.
Df Residuals:                     823   BIC:                            -3216.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const     